In [1]:
from pathlib import Path
import sys  

# Get my_package directory path from Notebook
parent_dir = str(Path().resolve().parents[0])
print(parent_dir)
# Add to sys.path

path_set = set(sys.path)
if parent_dir not in path_set:
    sys.path.insert(0, parent_dir)

print(sys.path)

C:\Users\clorch\Documents\education-benchmarking-and-insights\data-pipeline
['C:\\Users\\clorch\\Documents\\education-benchmarking-and-insights\\data-pipeline', 'c:\\Users\\clorch\\Documents\\education-benchmarking-and-insights\\data-pipeline\\workbooks', 'c:\\Users\\clorch\\AppData\\Local\\anaconda3\\python311.zip', 'c:\\Users\\clorch\\AppData\\Local\\anaconda3\\DLLs', 'c:\\Users\\clorch\\AppData\\Local\\anaconda3\\Lib', 'c:\\Users\\clorch\\AppData\\Local\\anaconda3', '', 'c:\\Users\\clorch\\AppData\\Local\\anaconda3\\Lib\\site-packages', 'c:\\Users\\clorch\\AppData\\Local\\anaconda3\\Lib\\site-packages\\win32', 'c:\\Users\\clorch\\AppData\\Local\\anaconda3\\Lib\\site-packages\\win32\\lib', 'c:\\Users\\clorch\\AppData\\Local\\anaconda3\\Lib\\site-packages\\Pythonwin']


# VMFI Data processing pipeline

This workbook aims to emulate the current data processing pipeline that occurs in VMFI pipeline. The logic and processing is largely based on the following document [Insights data portal - Data sources and sql analysis](https://educationgovuk.sharepoint.com.mcas.ms/:w:/r/sites/VMFI/_layouts/15/Doc.aspx?sourcedoc=%7B38C1DC37-7CDB-48B8-9E22-284F4F311C0B%7D&file=1.%20Insights%20portal%20-%20data%20sources%20and%20sql%20analysis%20v010%20-%20Copy.docx&action=default&mobileredirect=true) and will stay true to this document even if the existing stored procedures are doing something different. This will form the basis of a gap analysis going forward. 

All data loaded in the following workbook comes from the set of CSV files in the `data` folder alongside this workbook. These datasets are for the most part from the list at the start of the linked document. However, because there is additional standing data required to fully implement the pipeline then this data has been exported from the development VMFI pipeline database. These files are currently: 

| File name | DB Table |
|:----------|----------|
|standing_data_cdc.csv | standing_data.cdc |

In [2]:
import src.pipeline.pre_processing as pre_processing
import src.pipeline.output_schemas as output_schemas
import pandas as pd
import time
import glob
import os

In [3]:
# Create and clean directory
from pathlib import Path
Path("output/pre-processing").mkdir(parents=True, exist_ok=True)

files = glob.glob("output/pre-processing/*")
for f in files:
    os.remove(f)

In [4]:
start_time = time.time()
current_year = 2022

## CDC data load and preparation

School buildings condition dataset. Based on the surveys performed throughout 2018-2019.

The data in the file `data/standing_data_cdc.csv` is just an export of the data in `standing_data.cdc` table. Without the Year and Import ID fields. In future this will likely have to be read directly from the source database as per [this document.](https://educationgovuk.sharepoint.com.mcas.ms/:w:/r/sites/VMFI/_layouts/15/Doc.aspx?sourcedoc=%7B38C1DC37-7CDB-48B8-9E22-284F4F311C0B%7D&file=1.%20Insights%20portal%20-%20data%20sources%20and%20sql%20analysis%20v010%20-%20Copy.docx&action=default&mobileredirect=true) 

In [5]:
cdc = pre_processing.prepare_cdc_data('data/cdc.csv', current_year)

In [6]:
#cdc.to_csv('output/pre-processing/cdc.csv')
cdc

,Total Internal Floor Area,Age Average Score
URN,,
100150,2803.0,48.358188
100162,2105.0,133.162945
100164,2934.0,97.0
100166,2040.0,91.705882
105304,1602.0,35.752809
...,...,...
144913,3111.0,16.704275
144917,2620.0,78.412214
105623,3382.0,7.0


## School Census data load

*Pupil Census* - DfE data collection providing information about school and pupil characteristics, for example percentage of pupils claiming free school`z meals, or having English as their second language. 

*Workforce census* - Single reference for all school workforce statistics based on staff working in publicly funded schools in England.

The following code loads both the workforce and pupil census data and preforms an `inner` join by URN on the data sets.

In [7]:
census = pre_processing.prepare_census_data('data/census_workforce.xlsx', 'data/census_pupils.csv')

#todo - add logic for highest / lowest age from census data

In [8]:
#census.to_csv('output/pre-processing/census.csv')
census

,region_name,district_administrative_name,ward_name,Percentage claiming Free school meals,Percentage Free school meals,number of pupils whose first language is known or believed to be other than English,Total School Workforce (Headcount),Total Number of Teachers in the Leadership Group (Headcount),Total Number of Teachers (Headcount),Total Number of Teaching Assistants (Headcount),...,Total Number of Auxiliary Staff (Headcount),Total School Workforce (Full-Time Equivalent),Total Number of Teachers in the Leadership Group (Full-time Equivalent),Total Number of Teachers (Full-Time Equivalent),Total Number of Teaching Assistants (Full-Time Equivalent),FullTimeOther,Total Number of Auxiliary Staff (Full-Time Equivalent),Pupil: Teacher Ratio (Full-Time Equivalent of qualified and unqualified teachers),Teachers with Qualified Teacher Status (%) (Headcount),Number of Vacant Teacher Posts
URN,,,,,,,,,,,,,,,,,,,,,
141334,East Midlands,Nottingham,Bilborough,33.8,52.3,93.0,48,3,15,14,...,13,34.17,2.64,13.11,10.29,4.82,5.95,24.8,100.000000,0
141396,East Midlands,Nottingham,Aspley,23.4,60.3,236.0,118,4,39,34,...,34,82.47,4.00,34.00,29.55,10.13,8.79,18.3,100.000000,0
141397,East Midlands,Nottingham,Bilborough,33.2,47.7,127.0,105,5,27,42,...,27,72.81,4.24,24.55,31.84,6.55,9.87,19.7,100.000000,0
142223,East Midlands,Nottingham,Wollaton West,5.1,8.7,343.0,156,5,56,44,...,47,99.66,4.16,47.12,33.07,6.57,12.90,23.0,100.000000,0
144396,East Midlands,Nottingham,Bulwell,56.7,64.8,29.0,37,2,13,9,...,11,25.57,2.00,11.39,7.36,4.00,2.82,18.1,100.000000,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
104642,North West,Liverpool,Church,2.4,2.6,14.0,52,4,18,10,...,18,34.47,3.60,15.80,6.27,5.22,7.18,26.6,100.000000,0
104643,North West,Liverpool,Cressington,3.5,8.5,13.0,68,3,19,24,...,19,39.89,3.00,17.40,11.74,3.34,7.41,24.7,100.000000,0
104645,North West,Liverpool,Tuebrook and Stoneycroft,32.9,33.8,43.0,37,3,13,10,...,10,26.47,3.00,12.40,6.78,3.19,4.10,19.1,92.307692,0


## Special Education Needs (SEN) data load and preparation

Special educational needs dataset. Contains information about the number of pupils, who require various SEN provisions. This loads the `SEN` data, which originates from [here](https://explore-education-statistics.service.gov.uk/find-statistics/special-educational-needs-in-england#dataDownloads-1)

In [9]:
sen = pre_processing.prepare_sen_data('data/sen.csv')

In [10]:
#sen.to_csv("output/pre-processing/sen.csv")
sen

,Total pupils,EHC plan,Percentage SEN,Primary Need SPLD,Primary Need MLD,Primary Need SLD,Primary Need PMLD,Primary Need SEMH,Primary Need SLCN,Primary Need HI,...,Prov_SLD,Prov_PMLD,Prov_SEMH,Prov_SLCN,Prov_HI,Prov_VI,Prov_MSI,Prov_PD,Prov_ASD,Prov_OTH
URN,,,,,,,,,,,,,,,,,,,,,
100000,271,8,2.95203,2,4,0,0,9,31,2,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
100001,739,0,0.0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
100002,269,0,0.0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
100003,1045,0,0.0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
100005,136,2,1.470588,0,0,0,1,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
149557,41,3,7.317073,2,0,0,0,0,1,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
149632,1291,58,4.492641,31,15,0,0,20,25,8,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
149633,86,0,0.0,2,1,0,0,1,1,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## KS2 and KS4 processing

In [11]:
#ks2 = pre_processing.prepare_ks2_data('data/ks2.xlsx')
ks2 = pd.DataFrame({'URN':[100,200]})

In [12]:
#ks2.to_csv('output/pre-processing/ks2.csv')
ks2

,URN
0,100
1,200


In [13]:
#ks4 = pre_processing.prepare_ks4_data('data/ks4.xlsx')
ks4 = pd.DataFrame({'URN':[100,200]})

In [14]:
#ks4.to_csv('output/pre-processing/ks4.csv')
ks4

,URN
0,100
1,200


## AR Data load and preparation

This loads the Annual accounts return dataset and the corresponding mapping file. This extract only contains benchmarking section, which consists of submissions of costs, income, and balances of individual academies.

The mapping file, contains the mapping from AR4 cell references to cost categories and descriptions.

In [15]:
(trust_ar, academy_ar) = pre_processing.prepare_aar_data('data/academy_ar.xlsx')

In [16]:
#academy_ar.to_csv('output/pre-processing/academy_ar.csv')
academy_ar

,Academy Balance,Trust Balance,Central Services Balance,PFI School,Central Services Financial Position,Academy Financial Position,Trust Financial Position
academyupin,,,,,,,
111443,NaN,-1899000.0,-1830000.0,Non-PFI school,Deficit,Unknown,Deficit
111451,NaN,2880000.0,-8541000.0,Non-PFI school,Deficit,Unknown,Surplus
111453,NaN,1985000.0,-3191000.0,Non-PFI school,Deficit,Unknown,Surplus
111710,-1544000.0,NaN,0.0,Non-PFI school,Deficit,Deficit,Unknown
113087,NaN,-1232000.0,0.0,Non-PFI school,Deficit,Unknown,Deficit
...,...,...,...,...,...,...,...
164644,NaN,341000.0,-253000.0,Non-PFI school,Deficit,Unknown,Surplus
164745,NaN,1768000.0,-989000.0,Non-PFI school,Deficit,Unknown,Surplus
164746,NaN,5380000.0,-8019000.0,Non-PFI school,Deficit,Unknown,Surplus


In [17]:
#trust_ar.to_csv('output/pre-processing/trust_ar.csv')
trust_ar

,DFE/EFA Revenue grants (includes Coronavirus Government Funding,of which: Coronavirus Government Funding,SEN funding,Other DfE/EFA Revenue Grants,Other income - LA & other Government grants,"Government source, non-grant",Academies,Non-Government,All income from facilities and services,Income from catering,...,Total Staff Costs,Maintenance & Improvement Costs,Premises Costs,Catering Expenses,Occupation Costs,Total Costs of Supplies and Services,Total Costs of Educational Supplies,Costs of Brought in Professional Services,Total Expenditure,"Share of Revenue Reserve, distributed on per pupil basis\n"
trustupin,,,,,,,,,,,,,,,,,,,,,
122824,370608000.0,4806000.0,8930000.0,10300000.0,0.0,0.0,0.0,0.0,3727000.0,3194000.0,...,308877000.0,8659000.0,31303000.0,13153000.0,27121000.0,54864000.0,47174000.0,2113000.0,422255000.0,51443000.0
122836,221780000.0,2899000.0,14857000.0,2429000.0,0.0,0.0,1725000.0,1030000.0,1761000.0,3186000.0,...,163619000.0,1255000.0,12975000.0,9812000.0,19221000.0,24633000.0,14902000.0,7374000.0,220448000.0,21172000.0
124263,75303000.0,939000.0,2979000.0,0.0,2735000.0,0.0,0.0,1015000.0,330000.0,1096000.0,...,60897000.0,3255000.0,5341000.0,43000.0,1471000.0,12088000.0,4871000.0,2106000.0,79797000.0,1618000.0
126463,12052000.0,363000.0,45000.0,0.0,0.0,0.0,0.0,0.0,28000.0,194000.0,...,8323000.0,782000.0,1709000.0,613000.0,1125000.0,1760000.0,844000.0,428000.0,12917000.0,2754000.0
133307,25623000.0,539000.0,61000.0,1444000.0,0.0,0.0,0.0,611000.0,327000.0,538000.0,...,20407000.0,583000.0,1612000.0,1278000.0,2803000.0,3171000.0,1603000.0,853000.0,27993000.0,3294000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
163899,2002000.0,7000.0,38000.0,168000.0,0.0,0.0,0.0,9000.0,141000.0,0.0,...,1954000.0,26000.0,114000.0,73000.0,142000.0,154000.0,79000.0,34000.0,2364000.0,176000.0
164130,2029000.0,0.0,58000.0,80000.0,1000.0,0.0,0.0,22000.0,74000.0,0.0,...,2044000.0,32000.0,75000.0,69000.0,138000.0,300000.0,111000.0,147000.0,2557000.0,201000.0
164242,7441000.0,61000.0,268000.0,303000.0,1000.0,0.0,0.0,63000.0,158000.0,93000.0,...,6566000.0,93000.0,306000.0,395000.0,688000.0,670000.0,227000.0,258000.0,8230000.0,1618000.0


Create a summary table for the AR stance of each distinct academy in the table.

Now compute the trust financial position in the same manor as the individual academy position

## Academy and maintained schools data load and preparation

This reads the main GIAS data (edubasealldataYYYYMMDD file) and the associated links file (links_edubasealldataYYYYMMDD file). This is taken from the [GIAS Service](https://get-information-schools.service.gov.uk/help)

Other columns are tidied up by asserting the correct type for that column. This is tidying phase is largly because on load integer columns will be inferred to be a float as opposed to an integer.

In [18]:
schools = pre_processing.prepare_schools_data('data/gias.csv','data/gias_links.csv')


In [19]:
#schools.to_csv('output/pre-processing/schools.csv')
schools.sort_index()

,LA (code),LA (name),EstablishmentNumber,EstablishmentName,TypeOfEstablishment (code),TypeOfEstablishment (name),EstablishmentStatus (code),EstablishmentStatus (name),OpenDate,CloseDate,...,UrbanRural (name),BoardingEstablishment (name),PreviousLA (code),PreviousLA (name),PreviousEstablishmentNumber,OfstedRating (name),MSOA (code),LSOA (code),LA Establishment Number,HeadName
URN,,,,,,,,,,,,,,,,,,,,,
100000,201,City of London,3614,The Aldgate School,2,Voluntary aided school,1,Open,NaT,NaT,...,(England/Wales) Urban major conurbation,<NA>,999,<NA>,<NA>,Outstanding,E02000001,E01032739,201-3614,Miss Alexandra Allan
100001,201,City of London,6005,City of London School for Girls,11,Other independent school,1,Open,1920-01-01,NaT,...,(England/Wales) Urban major conurbation,Does not have boarders,999,<NA>,<NA>,,E02000001,E01000002,201-6005,Mrs Jenny Brown
100002,201,City of London,6006,St Paul's Cathedral School,11,Other independent school,1,Open,1939-01-01,NaT,...,(England/Wales) Urban major conurbation,Has boarders,999,<NA>,<NA>,,E02000001,E01032739,201-6006,<NA>
100003,201,City of London,6007,City of London School,11,Other independent school,1,Open,1919-01-01,NaT,...,(England/Wales) Urban major conurbation,Does not have boarders,999,<NA>,<NA>,,E02000001,E01032739,201-6007,Mr Alan Bird
100005,202,Camden,1048,Thomas Coram Centre,15,Local authority nursery school,1,Open,NaT,NaT,...,(England/Wales) Urban major conurbation,<NA>,999,<NA>,<NA>,Outstanding,E02007115,E01000937,202-1048,Ms Perina Holness
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
402468,679,Monmouthshire,5500,King Henry viii 3-19 School,30,Welsh establishment,1,Open,2023-09-01,NaT,...,<NA>,<NA>,999,<NA>,<NA>,,999999999,999999999,679-5500,<NA>
402469,681,Cardiff,2333,Ysgol Gynradd Groes-Wen Primary,30,Welsh establishment,1,Open,2023-09-01,NaT,...,(England/Wales) Rural village,<NA>,999,<NA>,<NA>,,W02000380,W01001729,681-2333,<NA>
402470,668,Pembrokeshire,2398,Ysgol Bro Penfro,30,Welsh establishment,4,Proposed to open,2024-09-01,NaT,...,(England/Wales) Rural town and fringe,<NA>,999,<NA>,<NA>,,W02000140,W01000607,668-2398,<NA>


Merge required GIAS, census, sen, cdc, PFI, and arr data with the base academy data

In [20]:
academies = pre_processing.build_academy_data('data/academy_master_list.csv', 
                                              current_year, schools, census, sen, cdc, 
                                              academy_ar, trust_ar, ks2, ks4)

In [21]:
academies.to_csv('output/pre-processing/academies.csv', columns=output_schemas.academies_output)
academies.sort_index()

,URN,Company Registration Number,Incorporation Date,Academy Trust UPIN,Trust UKPRN,Trust Name,Academy Name,Academy UPIN,Trust Type,Date Opened,...,Catering Expenses,Occupation Costs,Total Costs of Supplies and Services,Total Costs of Educational Supplies,Costs of Brought in Professional Services,Total Expenditure,"Share of Revenue Reserve, distributed on per pupil basis\n",Status,SchoolPhaseType,Finance Type
0,148853,10817580,2017-06-14 00:00:00.0000000,139821,10064612,1Excellence Multi Academy Trust,Evenwood Church of England Primary School,163480,Multi Academy Trust (MAT),2021-12-01 00:00:00.0000000,...,260000.0,496000.0,1046000.0,727000.0,158000.0,7172000.0,856000.0,(Re)opened in period,Primary,Academy
1,144542,10817580,2017-06-14 00:00:00.0000000,139821,10064612,1Excellence Multi Academy Trust,Pentland Primary School,138448,Multi Academy Trust (MAT),2017-07-01 00:00:00.0000000,...,260000.0,496000.0,1046000.0,727000.0,158000.0,7172000.0,856000.0,Open,Primary,Academy
2,144551,10817580,2017-06-14 00:00:00.0000000,139821,10064612,1Excellence Multi Academy Trust,St Mark's Church of England Primary School - S...,138465,Multi Academy Trust (MAT),2017-07-01 00:00:00.0000000,...,260000.0,496000.0,1046000.0,727000.0,158000.0,7172000.0,856000.0,Open,Primary,Academy
3,148854,10817580,2017-06-14 00:00:00.0000000,139821,10064612,1Excellence Multi Academy Trust,St Michael's Church of England Primary School ...,163504,Multi Academy Trust (MAT),2021-12-01 00:00:00.0000000,...,260000.0,496000.0,1046000.0,727000.0,158000.0,7172000.0,856000.0,(Re)opened in period,Primary,Academy
4,136730,07595434,2011-04-07 00:00:00.0000000,134890,10058682,5 Dimensions Trust,Shenley Brook End School,119734,Multi Academy Trust (MAT),2011-05-01 00:00:00.0000000,...,314000.0,1418000.0,2030000.0,1425000.0,141000.0,23966000.0,4087000.0,Open,Secondary,Academy
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10010,147031,07559293,2011-03-10 00:00:00.0000000,136062,10058621,Zenith Multi Academy Trust,Castle View School - Canvey Island,151873,Multi Academy Trust (MAT),2014-10-01 00:00:00.0000000,...,440000.0,1493000.0,3205000.0,2277000.0,227000.0,27548000.0,5685000.0,Open,Secondary,Academy
10011,145812,07559293,2011-03-10 00:00:00.0000000,136062,10058621,Zenith Multi Academy Trust,Laindon Park Primary School & Nursery,140554,Multi Academy Trust (MAT),2018-09-01 00:00:00.0000000,...,440000.0,1493000.0,3205000.0,2277000.0,227000.0,27548000.0,5685000.0,Open,Primary,Academy
10012,138865,07559293,2011-03-10 00:00:00.0000000,136062,10058621,Zenith Multi Academy Trust,The James Hornsby School,121878,Multi Academy Trust (MAT),2012-10-01 00:00:00.0000000,...,440000.0,1493000.0,3205000.0,2277000.0,227000.0,27548000.0,5685000.0,Open,Secondary,Academy
10013,136577,07559293,2011-03-10 00:00:00.0000000,136062,10058621,Zenith Multi Academy Trust,The King John School,119652,Multi Academy Trust (MAT),2011-04-01 00:00:00.0000000,...,440000.0,1493000.0,3205000.0,2277000.0,227000.0,27548000.0,5685000.0,Open,Secondary,Academy


Merge required census and cdc data to the maintained schools data set

In [22]:
# Load raw list from CSV
maintained_schools = pre_processing.build_maintained_school_data('data/maintained_schools_master_list.csv',current_year, schools, census, sen, cdc, ks2, ks4)

In [23]:
maintained_schools['LA Code']

URN
100000    201
100005    202
100006    202
100007    202
100008    202
         ... 
131818    341
132176    341
132793    341
132796    341
133332    341
Name: LA Code, Length: 10997, dtype: string

In [24]:
maintained_schools.to_csv('output/pre-processing/maintained_schools.csv', columns=output_schemas.maintained_schools_output)
#maintained_schools

In [25]:
# amalgamate school data

schools_table_output = [
   "UKPRN",
  "Establishment Name",
  "Finance Type",
  "Overall Phase",
  # "Ofsted Rating",
  "Ofsted Rating Description",
  "Ofsted Inspection Date",
  #  "Establishment Type",
  "LA Code",
  "LA Name",
  "Street",
  "Locality",
  "Address 3",
  "Town",
  "County",
  "Postcode",
  "Telephone",
  "Website",
  #  "Contact Email",
  #  "Headteacher email",
  "Head Teacher Name",
  "Trust Name",
  "Trust UKPRN",
  #  "Federation Name",
  #  "Federation Lead School UKPRN",
  "Has a sixth form",
  "Has a nursery",
  #  "Is PFI School",
  "Region",
  "District",
  "Ward",
  "London Weighting"
]



maintained_schools2 = maintained_schools.copy()
academies2 = academies.copy()
all_school_data = pd.concat((maintained_schools, academies))

#all_school_data
#all_school_data.to_csv('output/pre-processing/all_school_data.csv')

## Federation Capture




In [26]:
(hard_federations, soft_federations) = pre_processing.build_federations_data('data/gias_all_links.csv', maintained_schools)

In [27]:
hard_federations.to_csv('output/pre-processing/hard_federations.csv')
hard_federations

,LAEstab,FederationName,FederationUid,Closed Date,Members
URN,,,,,
100234,2042450,"The Viridis Federation of Orchard, Southwold &...",1652,<NA>,"[100234, 100242, 131141]"
100250,2042779,The LEAP Federation,17386,<NA>,"[100250, 100261, 130303]"
100258,2042864,New Wave Federation,1589,<NA>,"[100241, 100248, 100258]"
100263,2043358,Primary Advantage Federation,1473,<NA>,"[100224, 100225, 100232, 100263, 100266, 10026..."
101251,3021000,Barnet Early Years Alliance,5603,<NA>,"[101251, 101252, 101254]"
...,...,...,...,...,...
125246,9363928,Newlands CofE School Federation,17520,<NA>,"[125199, 125246]"
125288,9365206,The Federation of Holy Trinity and Pewley Down...,1025,<NA>,"[125288, 136755]"
125473,9367053,Federation of Manor Mead and Walton Leigh Schools,15744,<NA>,"[125468, 125473]"


In [28]:
soft_federations.to_csv('output/pre-processing/soft_federations.csv')
soft_federations[['LAEstab']]

,LAEstab
URN,
100391,2061104
100472,2071010
105861,3547006
108960,8012073
109417,8221009
109864,8692131
109964,8693026
111246,8965204
113172,8782249


### Timing Keep at the bottom

In [29]:
print(f'Processing Time: {time.time() - start_time} seconds')

Processing Time: 37.043379068374634 seconds
